## 표지판 예측 모델 학습

In [16]:
import the necessary packages
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.optimizers import Adam
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import argparse
import pickle
import cv2
import os

In [35]:
import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
sess  = tf.Session(config=config) 
keras.backend.set_session(sess)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

## 데이터세트 만들기

In [10]:
dir_url = './data/train/'

print( 'dir_url : {}'.format(dir_url) )
files = os.listdir(dir_url)
i = 1

print("[INFO] loading images...")
LABELS = set(["30","50","60"])
data = []
labels = []
train_path = './data/train'

for files_list in files : 
    print(files_list) # 30, 50, 60
    file = os.listdir(dir_url+files_list)
    for image in os.listdir(dir_url+files_list):
        imagepath = train_path + '/' + files_list + '/' + image
        
        img = cv2.imread(imagepath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(64,64) )
        data.append(img)
        
        if files_list not in LABELS:
            continue
        labels.append(files_list)
        

# convert the data and labels to NumPy arrays
print("[INFO] End loading images...")
data = np.array(data)
labels = np.array(labels)

data = data / 255.0 

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

dir_url : ./data/train/
[INFO] loading images...
50
60
30
[INFO] End loading images...


In [11]:
data.shape

(1960, 64, 64, 3)

In [12]:
lb.classes_

array(['30', '50', '60'], dtype='<U2')

In [13]:
# 데이터 분리
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

## 모델 생성

- 컨볼루션 레이어 : 입력 이미지 크기 64 x 64, 입력 이미지 채널 3개, 필터 크기 3 x 3, 필터 수 32개, 활성화 함수 ‘relu’
- 맥스풀링 레이어 : 풀 크기 2 x 2
- 드롭 레이어
- 컨볼루션 레이어 : 필터 크기 3 x 3, 필터 수 64개, 활성화 함수 ‘relu’
- 맥스풀링 레이어 : 풀 크기 2 x 2
- 드롭 레이어
- 플래튼 레이어
- 덴스 레이어 : 출력 뉴런 수 128개, 활성화 함수 ‘relu’
- 드롭 레이어
- 덴스 레이어 : 출력 뉴런 수 3개, 활성화 함수 ‘softmax’

In [8]:
model = Sequential()
model.add( Conv2D( 32, kernel_size=(3,3), activation='relu', input_shape=(64,64,3) ) )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )
model.add( Conv2D( 64, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )
model.add( Flatten() )
model.add( Dense(128, activation='relu') )
model.add( Dropout(0.25) )
model.add( Dense( 3, activation='softmax' ) )

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

#### 모델 컴파일
- [손실함수] 
- categorical_crossentropy 를 사용

- [최적화 함수]
- 1. 아담 
    -> opt = Adam(lr=1e-3, decay=1e-3 / 50)
- 2. SGD 
    -> opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
- 3. rmsprop

In [8]:
print("[INFO] compiling model...")
model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=['accuracy'] )

[INFO] compiling model...




In [36]:
# 모델 훈련
model.fit( trainX, trainY, batch_size=128, epochs= 50 , validation_split= 0.2 )

NameError: name 'model' is not defined

In [ ]:
# 모델 예측
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_) )

In [ ]:
#모델을 JSON 파일 형식으로 만들어 저장하기
model_json = model.to_json()
with open("./model/model.json", "w") as json_file : 
    json_file.write(model_json)

In [ ]:
# 모델 가중치(weights) 저장
model.save_weights('./model/first_try.h5')

## 학습된 모델을 사용하여 ROI영역 레이블 예측 및 그리기

In [ ]:
# import the necessary packages
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [ ]:
# 독일 이미지로 테스트
# 30, 50 , 60 : 3가지 이미지 데이터 
test_path_30 = './test_image/30_black.jpg'
test_path_50 = './test_image/50_29.jpg'
test_path_60 = './test_image/60_211.jpg'

# 예측 테스트
frame = cv2.imread(test_path_30)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (64, 64)).astype("float32")

# 예측
preds = model.predict(np.expand_dims(frame, axis=0))[0]
print(preds)

# 레이블 표시
label = lb.classes_[np.argmax(preds)]
text = "label text: {}".format(label)
print(text)

In [ ]:
# 전처리 과정을 거친 우리 데이터로 예측 테스트
img = cv2.imread('./test_image/sample3_515.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3, 3), 0)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1.5, 30, None, 570)

#print(circles) # (x, y, 반지름)
if circles is not None :
    circles = np.uint16(np.around(circles))
    for i in circles[0, :] :
        x = i[0]
        y = i[1]
        r = i[2]
        #print(x, y, r)
        #print((x-r, y-r), (x+r, y+r))
        cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
        cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 5)
        cv2.rectangle(img, (x-r, y-r), (x+r, y+r), (255, 0, 0), 1)
      #x,y 원의 중심 좌표 / r : 반지름  
frame = img[y-r:y+r, x-r:x+r]

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (64, 64)).astype("float32")


# 예측 후 , roi 영역 표시
preds = model.predict(np.expand_dims(frame, axis=0))[0]
print('label : {}'.format(preds))

#  레이블 확인
label = lb.classes_[np.argmax(preds)]
text = "predict: {}".format(label)
print(text)
cv2.putText(img, text, (x-50, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

# 이미지 확인
cv2.imshow('final', img)

# 종료 : 키보드 클릭
cv2.waitKey()
cv2.destroyAllWindows()
